In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D


In [2]:
mnist = tf.keras.datasets.mnist

(X_train,Y_train),(X_test,Y_test) = mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train[...,tf.newaxis]
X_test = X_test[...,tf.newaxis]

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
type(X_train[0])

numpy.ndarray

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,Y_train)).batch(32).shuffle(1000)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test,Y_test)).batch(32)

In [5]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(3, 3, activation="relu")
    self.flatten = Flatten()
    self.d1 = Dense(128, activation="relu")
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)
    return x

model = MyModel()



In [6]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.metrics.SparseCategoricalAccuracy()

In [8]:
@tf.function
def train(images,labels):
  with tf.GradientTape() as tape:
      pred = model(images)
      loss = loss_function(labels,pred)
  gradients = tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,pred)


In [9]:
@tf.function
def test(images, labels):
  pred = model(images)
  loss = loss_function(labels, pred)

  test_loss(loss)
  test_accuracy(labels, pred)

In [10]:
epochs = 12
for epoch in range(epochs):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()
  for images,labels in train_dataset:
    train(images,labels)


  for images,labels in test_dataset:
    test(images,labels)


  print("epoch:", epoch + 1,
          f"Train loss: {train_loss.result()}",
          f"Train accuracy: {train_accuracy.result()}",
          f"Test loss: {test_loss.result()}",
          f"test accuracy: {test_accuracy.result()}")



epoch: 1 Train loss: 0.2009309083223343 Train accuracy: 0.9419333338737488 Test loss: 0.09308649599552155 test accuracy: 0.972000002861023
epoch: 2 Train loss: 0.07725678384304047 Train accuracy: 0.9770833253860474 Test loss: 0.06728565692901611 test accuracy: 0.9793000221252441
epoch: 3 Train loss: 0.04969480261206627 Train accuracy: 0.9846500158309937 Test loss: 0.061353590339422226 test accuracy: 0.9804999828338623
epoch: 4 Train loss: 0.03488745167851448 Train accuracy: 0.9887999892234802 Test loss: 0.06398019194602966 test accuracy: 0.9815999865531921
epoch: 5 Train loss: 0.023523714393377304 Train accuracy: 0.9925333261489868 Test loss: 0.06906012445688248 test accuracy: 0.9790999889373779
epoch: 6 Train loss: 0.017201295122504234 Train accuracy: 0.9943333268165588 Test loss: 0.07567182928323746 test accuracy: 0.9796000123023987
epoch: 7 Train loss: 0.013582809828221798 Train accuracy: 0.9956666827201843 Test loss: 0.07253251224756241 test accuracy: 0.9811000227928162
epoch: 8 Tr

## Infrence

In [12]:
import cv2
import numpy as np

image = cv2.imread("/content/2.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.resize(image, (28, 28))
image = image[..., tf.newaxis]
image = image[tf.newaxis, ...]
image = image.astype("float32")

pred = model(image)
result = np.argmax(pred)
result

2

In [13]:
model.save("mnist_model")
